In [2]:
import glob
from tqdm.auto import tqdm, trange
import requests
import xml.etree.ElementTree as et
import json
import numpy as np
import re
import networkx as nx
import matplotlib.pyplot as plt
import random
import time

In [3]:
from html_parser import WykopParser

In [4]:
tag ='polska'
total_pages = 500


results = []
bar = trange(1, total_pages + 1)
for page_n in bar:
    success = False
    while not success:
        try:
            r = requests.get(f'https://www.wykop.pl/strona/{page_n}/').text
            success = True
        except:
            time.sleep(1)
    
    parser = WykopParser()
    parser.feed(r)
    
    articles = parser.root\
                     .find('div', [('class', 'article'), ('class', 'clearfix'), ('class', 'preview'), ('class', 'dC')])\
                     .contain('a', [('class', 'tag'), ('class', 'affect'), ('class', 'create')], [tag])
        
    titles= articles.find('h2')\
                    .find('a', [('href', 'http'), ('title', '')]) 
    texts = articles.find('p', [('class', 'text')])\
                    .find('a', [('href', 'http')])
    times = articles.find('time', [('datetime', '')])
    authrs= articles.find('div', [('class', 'fix-tagline')])\
                    .find('', [('class', 'color-'), ('class', 'affect')]).contain('em')
    
    if len(texts) != len(titles) or len(times) != len(titles) or len(authrs) != len(titles):
        print('broken page', len(titles), len(texts), len(times), len(authrs))
    
    for i, a1 in enumerate(titles):
        results.append((times[i].attrs['datetime'], a1.attrs['href'], 
                        authrs[i].data, a1.data, texts[i].data))
        
    bar.set_description('Scraped wykops: '+str(len(results)))
    

In [5]:
results[-1]

('2020-12-01T08:28:29+01:00',
 'https://www.wykop.pl/link/5827569/ksiadz-po-koledzie-w-tym-roku-nie-przyjdzie-jak-wplynie-to-na-finanse/',
 'janusz_z_czarnolasu',
 'Ksiądz po kolędzie w tym roku nie przyjdzie. Jak wpłynie to na finanse...',
 '\n\t\t\t\t\tW bogatych parafiach tzw. kolęda przynosiła co roku nawet kilkaset tysięcy złotych, w mniej zamożnych – kilkadziesiąt. Dla wielu wspólnot wpływy z "wizyt duszpasterskich" przynosiły jedną trzecią rocznych przychodów. Jak sobie teraz poradzą?\t\t        ')